# Problem Statement

Our aim is to create a game by generating a network of words based on levenstein distance. The goal of the game is to start with a word of a given length (e.g. a five letter word such as "boost") and turn it into another, predetermined word of the same length (e.g. brown) by changing one letter at a time. The goal is to get from your starting word to your end word in as few steps as possible.

This can be accomplished by generating a network using levenstein distance. Levenstein distance refers to how many changes it would take to turn one word into another. As an example, the words spoon and spool would have a levenstein distance of one. We can easily create a network in which every word is represented by a node. Nodes are then connected if they have a levenstein distance of one. Once this network is created, it is simple to plot the shortest path between any two words.

This network can then be used to generate interesting combinations of starting and ending words to test the player's skill.

In [5]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import jellyfish
from jellyfish import levenshtein_distance
import random
import sqlite3

## Importing Dictionaries

The dictionaries we are using are divided between initial letters. We will create a function to import each dictionary into a pandas dataframe.

In [14]:
# this function will create a dictionary of pandas dataframes, with each dataframe containing words which begin with the same letter
def get_dictionaries():
    letters = 'abcdefghijklmnopqrstuvwxyz'
    dictionary = {}
    # this will iterate through each letter of the alphabet
    for letter in letters:
        # adding each dataframe to a dictionary
        dictionary[f'{letter.upper()}_words'] = pd.read_csv(f'wordlist/{letter.upper()}word.csv', error_bad_lines=False, engine='python')
    return dictionary

In [104]:
# this function will create an edge list of all words of a given length
def make_edge_list(dictionary, length):
    words = []
    for letter in dictionary.values():
        for index, row in letter.iterrows():
            if len(row[0].strip()) == length:
                words.append(row[0].strip())

    word_set = set(four_letter_words)
    word_dict = list(dict.fromkeys(four_letter_words))                
                
    edge_dict = {}
    for word in word_dict:
        edge_dict[word] = []
        for subsequent_word in word_dict[word_dict.index(word):]:
            if jellyfish.levenshtein_distance(word, subsequent_word) == 1:
                edge_dict[word].append(subsequent_word)

        
    list1 = []
    list2 = []
    for key in edge_dict.keys():
        for value in edge_dict[key]:
            list1.append(key)
            list2.append(value)       
            
    list3 = []
    for a, b in zip(list1, list2):
        list3.append((a, b))           

    return np.array(list3)

# this function will take an edge list and convert it into a network
def make_graph(edge_list):
    G = nx.from_edgelist(edge_list)
    return G

In [108]:
# this function will choose two random words as the starting word and the target word
def gen_pair():
    while True:
        word1 = list(G.nodes)[random.randint(0, len(list(G.nodes)))]
        word2 = list(G.nodes)[random.randint(0, len(list(G.nodes)))]
        if (word1 != word2) and (word2 in nx.algorithms.descendants(G, word1)):
            return word1, word2
            break

In [109]:
# this is the main game function
# players will be given the starting word and be given the opportunity to input words with a levenstein distance of one away
# the game will provide hints to see if the player's guess is warmer or colder
def play():
    pair = gen_pair()
    source = pair[0]
    target = pair[1]
    print('Your starting word is {}'.format(pair[0]))
    print('Your target word is {}'.format(pair[1]))
    print('The shortest path between {} and {} is {} steps'.format(pair[0], pair[1], len(nx.shortest_path(G, source=pair[0], target=pair[1]))))
    print(nx.shortest_path(G, source=pair[0], target=pair[1]))
    previous = source
    current = ''
    while current != target:
        current = input('Enter next step: ')
        if current == '/quit':
            break
        elif levenshtein_distance(current, target) < levenshtein_distance(previous, target):
            print('previous guess: {}, current guess: {}, warmer'.format(previous, current))
            previous = current
        elif levenshtein_distance(current, target) > levenshtein_distance(previous, target):
            print('previous guess: {}, current guess: {}, colder'.format(previous, current))
            previous = current
        elif levenshtein_distance(current, target) == levenshtein_distance(previous, target):
            print('previous guess: {}, current guess: {}, same temp'.format(previous, current))
            previous = current
        

In [ ]:
play()